In [1]:
import sys
import os
import random
import pickle
import numpy as np
from preprocessing import make_delayed
from preprocessing import downsample_word_vectors

# Add the root project folder to sys.path (so ridge_utils becomes importable)
project_root = os.path.abspath('..')  # moves up from 'code/'
sys.path.append(project_root)

In [2]:
# Load the raw_text.pkl file
path_to_data = '/ocean/projects/mth240012p/shared/data'
with open(f'{path_to_data}/raw_text.pkl', 'rb') as f:
    raw_text = pickle.load(f)

print(type(raw_text))
print(len(raw_text)) # total 109 stories


<class 'dict'>
109


Before splitting the stories into train and test, since there are 8 more stories in the raw_text file than in the stories in subject 2 and 3, we exclude these stories since they cannot be used for both test or train. 

In [3]:
all_stories = set(raw_text.keys())
subj2_stories = set(os.path.splitext(f)[0] for f in os.listdir(f'{path_to_data}/subject2') if f.endswith('.npy'))
subj3_stories = set(os.path.splitext(f)[0] for f in os.listdir(f'{path_to_data}/subject3') if f.endswith('.npy'))
print(subj2_stories == subj3_stories) #fortunately, subject 2 and 3 has same stories
valid_stories = sorted(list(all_stories & subj2_stories & subj3_stories))
print(len(valid_stories)) 


True
101


Now we split test stories, train stories (3:7)

In [4]:
#we will just rename valid_stories as all_stories for the sake of simplicity.
all_stories = valid_stories
random.seed(42)
random.shuffle(all_stories)
split_idx = int(0.7 * len(all_stories))  # 70% for training

train_stories = all_stories[:split_idx]
test_stories = all_stories[split_idx:]

print(f"Train stories: {len(train_stories)}")
print(f"Test stories: {len(test_stories)}")
train_stories.sort()
test_stories.sort()

Train stories: 70
Test stories: 31


**Bag of Words Method**

1. For each story in train, we compute a matrix of one-hot vectors for each word in story.

Below, we first get a list of all the unique words in all the stories in train_stories. 

In [5]:
#compute and append all words in train_stories
allwords = []
for story in train_stories:
    temp_text = raw_text[story].data
    allwords += temp_text

print(len(allwords))

#compute unique words in train_stories
unique_words = list(set(allwords))
len(unique_words)


137105


10360

In [6]:
#compute and append all words in train_stories
allwords = []
for story in train_stories:
    temp_text = raw_text[story].data
    allwords += temp_text

print(len(allwords))

#compute unique words in train_stories
unique_words = list(set(allwords))
len(unique_words)


137105


10360

In [7]:
#make one-hot matrix for each story in training stories

word_to_index = {word: idx for idx, word in enumerate(unique_words)} #match each word to index
vocab_size = len(unique_words)

onehot_matrices = {}  # Storing one-hot matrix per story

for story in train_stories:
    words = raw_text[story].data
    num_words = len(words)
    
    # Initialize zero matrix
    onehot = np.zeros((num_words, vocab_size), dtype=np.int8)
    
    # Fill in the one-hot vectors
    for i, word in enumerate(words):
        if word in word_to_index:
            j = word_to_index[word]
            onehot[i, j] = 1
    
    onehot_matrices[story] = onehot #store matrix as value and story name as key

len(onehot_matrices) #70 keys == 70 train stories

70

In [8]:
#check if one-hot matrices are calculated well for first training story
for key, value in onehot_matrices.items():
    print("First key:", key)
    print(type(value))
    print("original shape of matrix for first story is:", value.shape) #row:number of total words in story
    #column: number of unique words in all training stories
    break


print("total length of words in first story is:", len(raw_text['adventuresinsayingyes'].data)) 
#number of rows of matrix matches the total length of words in first story

First key: adventuresinsayingyes
<class 'numpy.ndarray'>
original shape of matrix for first story is: (2309, 10360)
total length of words in first story is: 2309


2. Downsampling word vectors (one-hot matrices) to number of FMRIs taken for each story
Now we call downsample word vectors from the preprocessing.py to get the dimensions to match. 

In [9]:
#made dictionary whose keys are name of train story and values are raw_text[story]
wordseqs = {story: raw_text[story] for story in train_stories}

downsampled_vectors_1 = downsample_word_vectors(
    stories=train_stories,
    word_vectors=onehot_matrices,
    wordseqs=wordseqs
)


In [10]:
print(len(downsampled_vectors_1)) #matches the total number of training stories

#check the downsampled matrix of first two stories
i=0
for key, value in downsampled_vectors_1.items():
    print("First key:", key)
    print("downsampled matrix for first story is", value.shape)
    i += 1
    if i == 2: break
#we can see that row of the matrix shrunk from 1708 to 370

70
First key: adventuresinsayingyes
downsampled matrix for first story is (406, 10360)
First key: afatherscover
downsampled matrix for first story is (327, 10360)


In [11]:
trimmed_vectors_1 = {} #trimmed matrices for each train story 

for story, matrix in downsampled_vectors_1.items():
    trimmed_matrix = matrix[5:-10]
    trimmed_vectors_1[story] = trimmed_matrix

# check for first trimmed matrix, which corresponds to story 
for key, value in trimmed_vectors_1.items():
    print("First key:", key)
    print("trimmed matrix for first story is", value.shape)
    break


First key: adventuresinsayingyes
trimmed matrix for first story is (391, 10360)


In [12]:
#We can see that the number of rows of trimmed matrices matches
# the number of rows in the FMRI scan 
itsabox_s2 = np.load(f'{path_to_data}/subject2/adventuresinsayingyes.npy')

print("Type:", type(itsabox_s2))
print("Shape:", itsabox_s2.shape)

Type: <class 'numpy.ndarray'>
Shape: (391, 94251)


3. Create lagged versions of the features using make_delayed with delays ranging form [1, 4] inclusive

In [13]:
delayed_vectors_1 = {}

for story, trimmed_matrix in trimmed_vectors_1.items():
    X_lagged = make_delayed(trimmed_matrix, delays=[1, 2, 3, 4])
    delayed_vectors_1[story] = X_lagged

# check for first trimmed matrix, which corresponds to story 'penpal'
for key, value in delayed_vectors_1.items():
    print("First key:", key)
    print("delayed matrix for first story is", value.shape)
    break

First key: adventuresinsayingyes
delayed matrix for first story is (391, 41440)


Now we make delayed vectors for test stories as well.

In [14]:
vocab_size = len(unique_words) #unique words are from TRAIN STORIES

onehot_matrices_test = {}  # Storing one-hot matrix per story

for story in test_stories:
    words = raw_text[story].data
    num_words = len(words)
    
    # Initialize zero matrix
    onehot = np.zeros((num_words, vocab_size), dtype=np.int8)
    
    # Fill in the one-hot vectors
    for i, word in enumerate(words):
        if word in word_to_index:
            j = word_to_index[word]
            onehot[i, j] = 1
    
    onehot_matrices_test[story] = onehot #store matrix as value and story name as key

print(len(onehot_matrices_test)) #30 keys

wordseqs2 = {story: raw_text[story] for story in test_stories} #wordseqs for test stories
downsampled_vectors_1_test = downsample_word_vectors(
    stories=test_stories,
    word_vectors=onehot_matrices_test,
    wordseqs=wordseqs2
)

trimmed_vectors_1_test = {} #trimmed matrices for each test story 

for story, matrix in downsampled_vectors_1_test.items():
    trimmed_matrix = matrix[5:-10]
    trimmed_vectors_1_test[story] = trimmed_matrix

delayed_vectors_1_test = {}

for story, trimmed_matrix in trimmed_vectors_1_test.items():
    X_lagged = make_delayed(trimmed_matrix, delays=[1, 2, 3, 4])
    delayed_vectors_1_test[story] = X_lagged



31


In [15]:
import gc
del raw_text
gc.collect()

11

**PART2**

In [16]:
from ridge_utils.ridge import ridge_corr, ridge_corr_pred
import numpy as np
import logging
from ridge_utils.ridge import ridge_corr
from ridge_utils.utils import mult_diag
import random
from sklearn.model_selection import KFold

**Subject2**

In [17]:
path_s2 = '/ocean/projects/mth240012p/shared/data/subject2/'
Y_s2_train_dict = {}
for story in train_stories:
    Y_s2_train_dict[story] = np.load(path_s2 + f"{story}.npy")
Y_s2_test_dict = {}
for story in test_stories:
    Y_s2_test_dict[story] = np.load(path_s2 + f"{story}.npy")

# Stack X to make matrix
X_train_full = np.vstack([delayed_vectors_1[s] for s in train_stories]) 
X_test_full = np.vstack([delayed_vectors_1_test[s] for s in test_stories])    

# Stack Y2 to make one matrix
Y_train_s2 = np.vstack([Y_s2_train_dict[s] for s in train_stories])
Y_test_s2 = np.vstack([Y_s2_test_dict[s] for s in test_stories])

2(1) Fit ridge regression with alphas =1

In [23]:
logging.basicConfig(level=logging.INFO)
ridge_logger = logging.getLogger("ridge_corr")
def safe_zscore(v):
    mean = v.mean(0)
    std = v.std(0)
    std[std == 0] = 1 
    return (v - mean) / std

zs = safe_zscore
X_train_full = zs(np.nan_to_num(X_train_full))
X_test_full = zs(np.nan_to_num(X_test_full))
Y_train_s2 = zs(np.nan_to_num(Y_train_s2))
Y_test_s2 = zs(np.nan_to_num(Y_test_s2))

In [24]:
print(np.isnan(X_train_full).sum()) 
print(np.isnan(X_test_full).sum())
print(np.isnan(Y_train_s2).sum())
print(np.isnan(Y_test_s2).sum())

0
0
0
0


In [25]:
#subject 2
num_vox_s2 = 94251
corr_s2 = ridge_corr_pred(X_train_full, X_test_full, Y_train_s2, Y_test_s2, valphas= np.ones(num_vox_s2))
mean_cc_2 = np.mean(corr_s2[np.isfinite(corr_s2)]) #exclude
print("Mean CC for sub2:", mean_cc_2)
#We can see that when alphas are poorly set, the mean CC across voxels are very low. 

INFO:ridge_corr:Doing SVD...
INFO:ridge_corr:Dropped 70 tiny singular values.. (U is now (24955, 24885))
INFO:ridge_corr:Training stimulus has LSV norm: 644.162
/jet/home/tyang11/stat-214-lab3/lab3/ridge_utils/ridge.py:8: RuntimeWarning: invalid value encountered in divide
  zs = lambda v: (v-v.mean(0))/v.std(0) ## z-score function
INFO:ridge_corr:Average difference between actual & assumed Prespvar: -0.000


Mean CC for sub2: 0.002158777535075201


2(1). CV 

We use 3-fold CV to get best alphas for each voxel. 

In [26]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# 5-fold cross-validation on training stories
kf = KFold(n_splits=3, shuffle=True, random_state=42)
alphas = np.logspace(1, 4, 2)  #  2 values from 10 to 10000
Rcorrs_folds = []  # Store Rcorrs for each fold

for fold, (train_idx, val_idx) in enumerate(kf.split(train_stories)):
    print(f"Processing fold {fold + 1}/3 for BoW")

    # Get the story names for this fold
    train = [train_stories[i] for i in train_idx] 
    val = [train_stories[i] for i in val_idx]     

    # Stack X and Y for this fold
    X_train = np.vstack([delayed_vectors_1[s] for s in train]) 
    X_val = np.vstack([delayed_vectors_1[s] for s in val])      
    Y_train = np.vstack([Y_s2_train_dict[s] for s in train])    
    Y_val = np.vstack([Y_s2_train_dict[s] for s in val])       

    # Z-score 
    X_train = zs(X_train)
    X_val = zs(X_val)
    Y_train = zs(Y_train)
    Y_val = zs(Y_val)

    # Run ridge_corr on this fold
    Rcorrs = ridge_corr(X_train, X_val, Y_train, Y_val, alphas, use_corr=True)
    Rcorrs = np.array(Rcorrs)  # (10, 94251)
    Rcorrs_folds.append(Rcorrs)

# Average Rcorrs across folds
Rcorrs_folds = np.array(Rcorrs_folds)  # (3, 10, 94251)
Rcorrs_5foldmean = Rcorrs_folds.mean(axis=0)  # (10, 94251)
# Select best alpha for each voxel
best_alpha_idx_s2 = np.argmax(Rcorrs_5foldmean, axis=0)  # (94251,)
valphas_s2 = alphas[best_alpha_idx_s2]  # (94251,)


Processing fold 1/3 for BoW


INFO:ridge_corr:Doing SVD...
INFO:ridge_corr:Dropped 46 tiny singular values.. (U is now (16401, 16355))
INFO:ridge_corr:Training stimulus has LSV norm: 422.999
INFO:ridge_corr:Average difference between actual & assumed Prespvar: nan
INFO:ridge_corr:Training: alpha=10.000, mean corr=0.00023, max corr=0.05652, over-under(0.20)=0
INFO:ridge_corr:Training: alpha=10000.000, mean corr=0.00213, max corr=0.05903, over-under(0.20)=0


Processing fold 2/3 for BoW


INFO:ridge_corr:Doing SVD...
INFO:ridge_corr:Dropped 47 tiny singular values.. (U is now (16264, 16217))
INFO:ridge_corr:Training stimulus has LSV norm: 526.841
INFO:ridge_corr:Average difference between actual & assumed Prespvar: 0.000
INFO:ridge_corr:Training: alpha=10.000, mean corr=0.00091, max corr=0.05241, over-under(0.20)=0
INFO:ridge_corr:Training: alpha=10000.000, mean corr=0.00295, max corr=0.08136, over-under(0.20)=0


Processing fold 3/3 for BoW


INFO:ridge_corr:Doing SVD...
INFO:ridge_corr:Dropped 47 tiny singular values.. (U is now (17245, 17198))
INFO:ridge_corr:Training stimulus has LSV norm: 536.001
INFO:ridge_corr:Average difference between actual & assumed Prespvar: 0.000
INFO:ridge_corr:Training: alpha=10.000, mean corr=0.00238, max corr=0.06554, over-under(0.20)=0
INFO:ridge_corr:Training: alpha=10000.000, mean corr=0.00256, max corr=0.07010, over-under(0.20)=0


In [27]:
#given valphas, we calculate test CCs on test stories. 
final_ccs_s2 = ridge_corr_pred(X_train_full, X_test_full, Y_train_s2, Y_test_s2, valphas_s2)

valid_ccs_s2 = final_ccs_s2[np.isfinite(final_ccs_s2)]  # Exclude NaNs/infs
mean_cc_s2 = np.mean(valid_ccs_s2)
median_cc_s2 = np.median(valid_ccs_s2)
top_1_percentile_cc_s2 = np.percentile(valid_ccs_s2, 99)  # Top 1 percentile
top_5_percentile_cc_s2 = np.percentile(valid_ccs_s2, 95)  # Top 5 percentile

# Print statistics
print(f"Mean test CC for subject 2 (BoW): {mean_cc_s2}")
print(f"Median test CC for subject 2 (BoW): {median_cc_s2}")
print(f"Top 1 percentile test CC for subject 2 (BoW): {top_1_percentile_cc_s2}")
print(f"Top 5 percentile test CC for subject 2 (BoW): {top_5_percentile_cc_s2}")


INFO:ridge_corr:Doing SVD...
INFO:ridge_corr:Dropped 70 tiny singular values.. (U is now (24955, 24885))
INFO:ridge_corr:Training stimulus has LSV norm: 644.162
/jet/home/tyang11/stat-214-lab3/lab3/ridge_utils/ridge.py:8: RuntimeWarning: invalid value encountered in divide
  zs = lambda v: (v-v.mean(0))/v.std(0) ## z-score function
INFO:ridge_corr:Average difference between actual & assumed Prespvar: -0.000


Mean test CC for subject 2 (BoW): 0.004206003832647905
Median test CC for subject 2 (BoW): 0.0037176974845577293
Top 1 percentile test CC for subject 2 (BoW): 0.036475893286999496
Top 5 percentile test CC for subject 2 (BoW): 0.024439891364196845


In [28]:
import gc
del Y_s2_train_dict
del Y_s2_test_dict
del Y_test_s2
del Y_train_s2
del Rcorrs_folds, Rcorrs_5foldmean

gc.collect()

178

**subject3**

In [29]:
path_s3 = '/ocean/projects/mth240012p/shared/data/subject3/'
Y_s3_train_dict = {}
for story in train_stories:
    Y_s3_train_dict[story] = np.load(path_s3 + f"{story}.npy")
Y_s3_test_dict = {}
for story in test_stories:
    Y_s3_test_dict[story] = np.load(path_s3 + f"{story}.npy")

In [30]:
#Stack Y3 to make one matrix
Y_train_s3 = np.vstack([Y_s3_train_dict[s] for s in train_stories])
Y_test_s3 = np.vstack([Y_s3_test_dict[s] for s in test_stories])
Y_train_s3 = zs(np.nan_to_num(Y_train_s3))
Y_test_s3 = zs(np.nan_to_num(Y_test_s3))

2(1). fit ridge for alpha = 1

In [31]:
#subject3
num_vox_s3 = 95556
corr_s3 = ridge_corr_pred(X_train_full, X_test_full, Y_train_s3, Y_test_s3, valphas= np.ones(num_vox_s3))
mean_cc_3 = np.mean(corr_s3[np.isfinite(corr_s3)]) #exclude
print("Mean CC for sub3:", mean_cc_3)

INFO:ridge_corr:Doing SVD...
INFO:ridge_corr:Dropped 70 tiny singular values.. (U is now (24955, 24885))
INFO:ridge_corr:Training stimulus has LSV norm: 644.162
INFO:ridge_corr:Average difference between actual & assumed Prespvar: -0.000


Mean CC for sub3: 0.0017076314413972662


2(2). CV

In [32]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# 5-fold cross-validation on training stories
kf = KFold(n_splits=3, shuffle=True, random_state=42)
alphas = np.logspace(1, 4, 2) 
Rcorrs_folds = []  # Reset for subject 3

for fold, (train_idx, val_idx) in enumerate(kf.split(train_stories)):
    print(f"Processing fold {fold + 1}/3 for BoW (subject 3)")

    train = [train_stories[i] for i in train_idx] 
    val = [train_stories[i] for i in val_idx]

    # Stack X and Y for this fold
    X_train = np.vstack([delayed_vectors_1[s] for s in train]) 
    X_val = np.vstack([delayed_vectors_1[s] for s in val])      
    Y_train = np.vstack([Y_s3_train_dict[s] for s in train])    
    Y_val = np.vstack([Y_s3_train_dict[s] for s in val])       

    # Z-score 
    X_train = zs(X_train)
    X_val = zs(X_val)
    Y_train = zs(Y_train)
    Y_val = zs(Y_val)

    # Run ridge_corr on this fold
    Rcorrs = ridge_corr(X_train, X_val, Y_train, Y_val, alphas, use_corr=True)
    Rcorrs = np.array(Rcorrs)  # (10, 94251)
    Rcorrs_folds.append(Rcorrs)

# Average Rcorrs across folds
Rcorrs_folds = np.array(Rcorrs_folds)  # (3, 10, 95556)
Rcorrs_5foldmean = Rcorrs_folds.mean(axis=0)  # (10, 95556)

# Select best alpha for each voxel
best_alpha_idx_s3 = np.argmax(Rcorrs_5foldmean, axis=0)  # (95556,)
valphas_s3 = alphas[best_alpha_idx_s3]  # (95556,)


Processing fold 1/3 for BoW (subject 3)


INFO:ridge_corr:Doing SVD...
INFO:ridge_corr:Dropped 46 tiny singular values.. (U is now (16401, 16355))
INFO:ridge_corr:Training stimulus has LSV norm: 422.999
INFO:ridge_corr:Average difference between actual & assumed Prespvar: -0.000
INFO:ridge_corr:Training: alpha=10.000, mean corr=0.00152, max corr=0.05325, over-under(0.20)=0
INFO:ridge_corr:Training: alpha=10000.000, mean corr=0.00518, max corr=0.07571, over-under(0.20)=0


Processing fold 2/3 for BoW (subject 3)


INFO:ridge_corr:Doing SVD...
INFO:ridge_corr:Dropped 47 tiny singular values.. (U is now (16264, 16217))
INFO:ridge_corr:Training stimulus has LSV norm: 526.841
INFO:ridge_corr:Average difference between actual & assumed Prespvar: 0.000
INFO:ridge_corr:Training: alpha=10.000, mean corr=0.00369, max corr=0.08274, over-under(0.20)=0
INFO:ridge_corr:Training: alpha=10000.000, mean corr=0.00565, max corr=0.10215, over-under(0.20)=0


Processing fold 3/3 for BoW (subject 3)


INFO:ridge_corr:Doing SVD...
INFO:ridge_corr:Dropped 47 tiny singular values.. (U is now (17245, 17198))
INFO:ridge_corr:Training stimulus has LSV norm: 536.001
INFO:ridge_corr:Average difference between actual & assumed Prespvar: 0.000
INFO:ridge_corr:Training: alpha=10.000, mean corr=0.00294, max corr=0.08075, over-under(0.20)=0
INFO:ridge_corr:Training: alpha=10000.000, mean corr=0.00636, max corr=0.10034, over-under(0.20)=0


In [33]:
# Given valphas, calculate test CCs on test stories
final_ccs_s3 = ridge_corr_pred(X_train_full, X_test_full, Y_train_s3, Y_test_s3, valphas_s3, use_corr=True)

# Compute statistics for subject 3
valid_ccs_s3 = final_ccs_s3[np.isfinite(final_ccs_s3)]  # Exclude NaNs/infs
mean_cc_s3 = np.mean(valid_ccs_s3)
median_cc_s3 = np.median(valid_ccs_s3)
top_1_percentile_cc_s3 = np.percentile(valid_ccs_s3, 99)  # Top 1 percentile
top_5_percentile_cc_s3 = np.percentile(valid_ccs_s3, 95)  # Top 5 percentile

# Print statistics
print(f"Mean test CC for subject 3 (BoW): {mean_cc_s3}")
print(f"Median test CC for subject 3 (BoW): {median_cc_s3}")
print(f"Top 1 percentile test CC for subject 3 (BoW): {top_1_percentile_cc_s3}")
print(f"Top 5 percentile test CC for subject 3 (BoW): {top_5_percentile_cc_s3}")

INFO:ridge_corr:Doing SVD...
INFO:ridge_corr:Dropped 70 tiny singular values.. (U is now (24955, 24885))
INFO:ridge_corr:Training stimulus has LSV norm: 644.162
INFO:ridge_corr:Average difference between actual & assumed Prespvar: -0.000


Mean test CC for subject 3 (BoW): 0.004901137088664349
Median test CC for subject 3 (BoW): 0.003983045173689538
Top 1 percentile test CC for subject 3 (BoW): 0.04508222720607162
Top 5 percentile test CC for subject 3 (BoW): 0.02705549544698708
